In [9]:
# !pip install 'sagemaker[local]' sagemaker --upgrade 

In [2]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

image_tag = 'test-autogluon'
sess = sagemaker.Session()
role = 'arn:aws:iam::514965996716:role/service-role/AmazonSageMaker-ExecutionRole-20181229T135829'# get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name


In [3]:
training_data = 's3://autogluon/datasets/Inc/train.csv'
test_data = 's3://autogluon/datasets/Inc/test.csv'

In [10]:
# image = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_tag}:latest'
image = image_tag

# this is just a dummy location. The model is called with train data. We use the current notebook as dummy train data.
uri = sess.upload_data('./deploy_model.ipynb')

artifacts = 's3://<your-bucket>/artifacts'
sm_model = Estimator(image, role, 
                     train_instance_count=1, 
                     train_instance_type='ml.m5.large', 
                     sagemaker_session=sess
                    )

In [11]:
# Run the train program because it is expected
sm_model.fit(
    {'training': training_data, 'testing': test_data}
)

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Invalid training image. Please provide a valid Amazon Elastic Container Registry path of the Docker image to run.